# **Imports**

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

import os
import PIL.Image as Image
from IPython.display import display

from sklearn.metrics import confusion_matrix, classification_report
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **Global Variables**

In [2]:
dataset_path = '/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data'
image_size = (400, 400)

# **Preprocessing**

In [3]:
train_tfms = transforms.Compose([transforms.Resize(image_size),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomRotation(15),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_tfms = transforms.Compose([transforms.Resize(image_size),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_time_tfms = transforms.Compose([transforms.Resize(image_size),
                                transforms.ToTensor(),
                                transforms.RandomRotation(90),
                                transforms.RandomHorizontalFlip(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


train_dataset = torchvision.datasets.ImageFolder(root=os.path.join(dataset_path, 'train'), transform = train_tfms)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 32, shuffle=True, num_workers = 2)

test_dataset = torchvision.datasets.ImageFolder(root=os.path.join(dataset_path, 'test'), transform = test_tfms)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 32, shuffle=False, num_workers = 2)

# **ResNet Model**

In [4]:
# i will use pretrained model for more accurcy
res_model = models.resnet34(pretrained=True)
num_ftrs = res_model.fc.in_features
num_classes = len(train_dataset.classes)

res_model.fc = nn.Linear(num_ftrs, num_classes)
res_model = res_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(res_model.parameters(), lr=0.01, momentum=0.9)

# to get more acc we will tracking learning rate 
lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold = 0.9)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 150MB/s] 


In [ ]:
epochs = 10  

train_losses = []
val_losses = []
accuracies = []

for epoch in range(epochs):
    # Training loop
    res_model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = res_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    train_loss = running_loss / len(train_dataset)
    train_losses.append(train_loss)
    
    res_model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = res_model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item() * inputs.size(0)
            
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_loss = val_loss / len(test_dataset)
    val_losses.append(val_loss)
    accuracy = correct / total
    accuracies.append(accuracy)
    

    print(f'Epoch {epoch+1}/{epochs}, '
          f'Train Loss: {train_loss:.4f}, '
          f'Val Loss: {val_loss:.4f}, '
          f'Val Accuracy: {accuracy:.4f}')
    

    lrscheduler.step(accuracy)

Epoch 1/10, Train Loss: 3.9779, Val Loss: 2.6616, Val Accuracy: 0.2996
Epoch 2/10, Train Loss: 1.6344, Val Loss: 1.6936, Val Accuracy: 0.5415
Epoch 3/10, Train Loss: 0.8619, Val Loss: 1.2549, Val Accuracy: 0.6432


In [ ]:
# Plotting the results
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, epochs + 1), train_losses, label='Training Loss')
plt.plot(range(1, epochs + 1), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, epochs + 1), accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# save model 
torch.save(res_model.state_dict(), 'resnet_model.pth')

# **Load and Prediction**

In [ ]:
loaded_model = models.resnet34(pretrained=False) 
loaded_model.fc = nn.Linear(loaded_model.fc.in_features, num_classes) 
loaded_model.load_state_dict(torch.load('resnet_model.pth'))
loaded_model.eval() 

def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((400, 400)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)
    return image

image_path = '/kaggle/input/test-data/Acura_ILX_2013_28_16_110_15_4_70_55_179_39_FWD_5_4_4dr_UMh.jpg'
input_image = preprocess_image(image_path)

with torch.no_grad():
    output = loaded_model(input_image)
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    predicted_class = torch.argmax(probabilities).item()

class_labels = train_dataset.classes
predicted_label = class_labels[predicted_class]

print(f"Predicted class: {predicted_label}")